## Downloading data as json

In [5]:
from decouple import config, AutoConfig
from datetime import datetime, timedelta
import os
import requests
from pathlib import Path

In [6]:
config = AutoConfig(search_path='.env') # <-- .env file located next to manage.py

In [7]:
# download the json by supplying the api token in the header
def get_json(endpoint, headers):
    """download the json by supplying the api token in the header"""
    headers['Accept'] = 'application/json' # csv?
    # pull_date = (datetime.now() - timedelta(days=180)).strftime("%Y-%m-%dT%H:%M:%S") # year, month, day, hour, minute, seconds, microseconds
    combined = []
    offset, counter = 0, 1
    error = False
    params = f"""$query=SELECT:*,* ORDER BY :id LIMIT 2000"""
    # response has two parts .json() and .headers https://www.w3schools.com/python/ref_requests_response.asp
    response = requests.get(endpoint, headers=headers, params=params)
    captured = response.json()
    combined.extend(captured)
    print('get_json complete')
    return combined

In [8]:
# Sodu API Credentials
API_TOKEN = config("API_TOKEN")
API_KEY_ID = config("API_KEY_ID")
API_KEY_SECRET = config("API_KEY_SECRET")

source_path_json = '/home/sanyashireen/sf_eviction/data_eviction/2023/3/26/api_raw_eviction_2023-03-26.json'
data_dir = Path('/home/sanyashireen/sf_eviction/data_eviction/2023/3/26')
data_dir.mkdir(parents=True, exist_ok=True)

SODA_url = 'https://data.sfgov.org/resource/5cei-gny5'
SODA_headers = {
    'keyId': API_KEY_ID,
    'keySecret': API_KEY_SECRET
}
content = get_json(SODA_url, SODA_headers)


get_json complete


In [3]:
source_path_json = '/home/sanyashireen/sf_eviction/data_eviction/2023/3/26/api_raw_eviction_2023-03-26.json'

In [6]:
type(content)

list

In [138]:
print(type(content[1]))

<class 'dict'>


## TESTING: Csv is corrupting the data so trying to write the data to pyspark directly

In [9]:
# testing writing the list of dictionaries (response.json()) returned by the API to a file locally as json file
# using json.dump
import json
out_file = open(source_path_json,"w", encoding='utf8')
json.dump(content, out_file, indent=4)
out_file.close()


## TESTING: If Pyspark can read the json data stored locally without corropting it

In [16]:
#spark.stop()

In [17]:
# Creating and testing script to clean and transform csv data using pyspark
# spark related packages

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]") \
    .appName('test') \
    .getOrCreate()


In [10]:
source_path_json = '/home/sanyashireen/sf_eviction/data_eviction/2023/3/26/raw_eviction_2023-03-26.json'
df = spark.read.option("multiline","true").json(str(source_path_json))

In [ ]:
# resting if pyspark can directly read the json data
source_path_json = '/home/sanyashireen/sf_eviction/data_eviction/2023/3/26/raw_eviction_2023-03-26.json'
df = spark.read.option("multiline","true").json(str(source_path_json))

In [11]:
df.count()

177578

In [12]:
df.printSchema()

root
 |-- :@computed_region_26cr_cadq: string (nullable = true)
 |-- :@computed_region_6ezc_tdp2: string (nullable = true)
 |-- :@computed_region_6pnf_4xz7: string (nullable = true)
 |-- :@computed_region_6qbp_sg9q: string (nullable = true)
 |-- :@computed_region_9jxd_iqea: string (nullable = true)
 |-- :@computed_region_ajp5_b2md: string (nullable = true)
 |-- :@computed_region_bh8s_q3mv: string (nullable = true)
 |-- :@computed_region_fyvs_ahh9: string (nullable = true)
 |-- :@computed_region_h4ep_8xdi: string (nullable = true)
 |-- :@computed_region_jwn9_ihcz: string (nullable = true)
 |-- :@computed_region_p5aj_wyqh: string (nullable = true)
 |-- :@computed_region_pigm_ib2e: string (nullable = true)
 |-- :@computed_region_qgnn_b9vv: string (nullable = true)
 |-- :@computed_region_rxqg_mtj9: string (nullable = true)
 |-- :@computed_region_yftq_j783: string (nullable = true)
 |-- :created_at: string (nullable = true)
 |-- :id: string (nullable = true)
 |-- :updated_at: string (nullab

In [13]:
df.head(3)

23/03/27 14:34:00 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[Row(:@computed_region_26cr_cadq='2', :@computed_region_6ezc_tdp2=None, :@computed_region_6pnf_4xz7='2', :@computed_region_6qbp_sg9q='53', :@computed_region_9jxd_iqea=None, :@computed_region_ajp5_b2md='20', :@computed_region_bh8s_q3mv='28859', :@computed_region_fyvs_ahh9='19', :@computed_region_h4ep_8xdi=None, :@computed_region_jwn9_ihcz='53', :@computed_region_p5aj_wyqh='4', :@computed_region_pigm_ib2e=None, :@computed_region_qgnn_b9vv='3', :@computed_region_rxqg_mtj9='7', :@computed_region_yftq_j783='2', :created_at='2023-01-23T23:46:16.858Z', :id='row-ufzj_22gk~drek', :updated_at='2023-01-23T23:46:33.925Z', :version='rv-wryr~aezj~xte7', access_denial=False, address='2500 Block Of Folsom  Street', breach=False, capital_improvement=False, city='San Francisco', client_location=Row(human_address='{"address": "", "city": "", "state": "", "zip": ""}', latitude='37.75649855484188', longitude='-122.41446453496935'), condo_conversion=False, constraints_date=None, demolition=False, developmen

In [156]:
df.show()

+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+--------------------+------------------+--------------------+-----------------+-------------+--------------------+------+-------------------+-------------+--------------------+----------------+----------------+----------+-----------+--------------------+-----------+-----------------------+--------------------+-------------------+-----------+-------------+----------------+--------------------+-----------+--------+-----------+-------------+------------------+--------------------+-----+-----------------+-------------------+--------------------+-----+
|:@computed_region_26cr_c

In [31]:
df.columns

[':@computed_region_26cr_cadq',
 ':@computed_region_6ezc_tdp2',
 ':@computed_region_6pnf_4xz7',
 ':@computed_region_6qbp_sg9q',
 ':@computed_region_9jxd_iqea',
 ':@computed_region_ajp5_b2md',
 ':@computed_region_bh8s_q3mv',
 ':@computed_region_fyvs_ahh9',
 ':@computed_region_h4ep_8xdi',
 ':@computed_region_jwn9_ihcz',
 ':@computed_region_p5aj_wyqh',
 ':@computed_region_pigm_ib2e',
 ':@computed_region_qgnn_b9vv',
 ':@computed_region_rxqg_mtj9',
 ':@computed_region_yftq_j783',
 ':created_at',
 ':id',
 ':updated_at',
 ':version',
 'access_denial',
 'address',
 'breach',
 'capital_improvement',
 'city',
 'client_location',
 'condo_conversion',
 'constraints_date',
 'demolition',
 'development',
 'ellis_act_withdrawal',
 'eviction_id',
 'failure_to_sign_renewal',
 'file_date',
 'good_samaritan_ends',
 'illegal_use',
 'late_payments',
 'lead_remediation',
 'neighborhood',
 'non_payment',
 'nuisance',
 'other_cause',
 'owner_move_in',
 'roommate_same_unit',
 'shape',
 'state',
 'substantial_r

In [14]:
len(df.columns)

49

In [157]:
print(f'The pyspark df is a list of records where each record is of type {type(df.head(1)[0])} \n\n {df.head(1)[0]}')

The pyspark df is a list of records where each record is of type <class 'pyspark.sql.types.Row'> 

 Row(:@computed_region_26cr_cadq='2', :@computed_region_6ezc_tdp2=None, :@computed_region_6pnf_4xz7='2', :@computed_region_6qbp_sg9q='53', :@computed_region_9jxd_iqea=None, :@computed_region_ajp5_b2md='20', :@computed_region_bh8s_q3mv='28859', :@computed_region_fyvs_ahh9='19', :@computed_region_h4ep_8xdi=None, :@computed_region_jwn9_ihcz='53', :@computed_region_p5aj_wyqh='4', :@computed_region_pigm_ib2e=None, :@computed_region_qgnn_b9vv='3', :@computed_region_rxqg_mtj9='7', :@computed_region_yftq_j783='2', :created_at='2023-01-23T23:46:16.858Z', :id='row-ufzj_22gk~drek', :updated_at='2023-01-23T23:46:33.925Z', :version='rv-wryr~aezj~xte7', access_denial=False, address='2500 Block Of Folsom  Street', breach=False, capital_improvement=False, city='San Francisco', client_location=Row(human_address='{"address": "", "city": "", "state": "", "zip": ""}', latitude='37.75649855484188', longitude=

In [15]:
print(f'The pyspark df is a list of records where each record is of type {type(df.head(1)[0])} \n\n {df.head(1)[0]}')

The pyspark df is a list of records where each record is of type <class 'pyspark.sql.types.Row'> 

 Row(:@computed_region_26cr_cadq='2', :@computed_region_6ezc_tdp2=None, :@computed_region_6pnf_4xz7='2', :@computed_region_6qbp_sg9q='53', :@computed_region_9jxd_iqea=None, :@computed_region_ajp5_b2md='20', :@computed_region_bh8s_q3mv='28859', :@computed_region_fyvs_ahh9='19', :@computed_region_h4ep_8xdi=None, :@computed_region_jwn9_ihcz='53', :@computed_region_p5aj_wyqh='4', :@computed_region_pigm_ib2e=None, :@computed_region_qgnn_b9vv='3', :@computed_region_rxqg_mtj9='7', :@computed_region_yftq_j783='2', :created_at='2023-01-23T23:46:16.858Z', :id='row-ufzj_22gk~drek', :updated_at='2023-01-23T23:46:33.925Z', :version='rv-wryr~aezj~xte7', access_denial=False, address='2500 Block Of Folsom  Street', breach=False, capital_improvement=False, city='San Francisco', client_location=Row(human_address='{"address": "", "city": "", "state": "", "zip": ""}', latitude='37.75649855484188', longitude=

In [7]:
# from web data
# Extracting column names to decide which ones are of interest
# Convert type row to dictionary
#row_as_dict = df.head(1)[0].asDict()

# Column Names
#print(list(row_as_dict.keys()))

# Sample Column and Value
#for key, value in row_as_dict.items():
    #print(f'{key}={value}')


['eviction_id', 'address', 'city', 'state', 'zip', 'file_date', 'non_payment', 'breach', 'nuisance', 'illegal_use', 'failure_to_sign_renewal', 'access_denial', 'unapproved_subtenant', 'owner_move_in', 'demolition', 'capital_improvement', 'substantial_rehab', 'ellis_act_withdrawal', 'condo_conversion', 'roommate_same_unit', 'other_cause', 'late_payments', 'lead_remediation', 'development', 'good_samaritan_ends', 'constraints_date', 'supervisor_district', 'neighborhood', 'client_location', 'shape', ':@computed_region_6qbp_sg9q', ':@computed_region_qgnn_b9vv', ':@computed_region_26cr_cadq', ':@computed_region_ajp5_b2md', ':@computed_region_fyvs_ahh9', ':@computed_region_p5aj_wyqh', ':@computed_region_rxqg_mtj9', ':@computed_region_yftq_j783', ':@computed_region_bh8s_q3mv', ':@computed_region_9jxd_iqea', ':@computed_region_6ezc_tdp2', ':@computed_region_6pnf_4xz7', ':@computed_region_h4ep_8xdi', ':@computed_region_pigm_ib2e', ':@computed_region_jwn9_ihcz']
eviction_id=M222138
address=400 B

In [36]:
# Extracting column names to decide which ones are of interest
# Convert type row to dictionary
row_as_dict = df.head(1)[0].asDict()

# Column Names
print(list(row_as_dict.keys()))

# Sample Column and Value
for key, value in row_as_dict.items():
    print(f'{key}={value}')
    
# new cols updated_at, id, created_at, version  missing when extracting from the URL

[':@computed_region_26cr_cadq', ':@computed_region_6ezc_tdp2', ':@computed_region_6pnf_4xz7', ':@computed_region_6qbp_sg9q', ':@computed_region_9jxd_iqea', ':@computed_region_ajp5_b2md', ':@computed_region_bh8s_q3mv', ':@computed_region_fyvs_ahh9', ':@computed_region_h4ep_8xdi', ':@computed_region_jwn9_ihcz', ':@computed_region_p5aj_wyqh', ':@computed_region_pigm_ib2e', ':@computed_region_qgnn_b9vv', ':@computed_region_rxqg_mtj9', ':@computed_region_yftq_j783', ':created_at', ':id', ':updated_at', ':version', 'access_denial', 'address', 'breach', 'capital_improvement', 'city', 'client_location', 'condo_conversion', 'constraints_date', 'demolition', 'development', 'ellis_act_withdrawal', 'eviction_id', 'failure_to_sign_renewal', 'file_date', 'good_samaritan_ends', 'illegal_use', 'late_payments', 'lead_remediation', 'neighborhood', 'non_payment', 'nuisance', 'other_cause', 'owner_move_in', 'roommate_same_unit', 'shape', 'state', 'substantial_rehab', 'supervisor_district', 'unapproved_sub

### Analysis
* As we can see the columns of interest are not all columns.
* The last columns name `@computed_region*` is not of interest so we will ignore those columns.
* And will extract only those columns from the raw dataset
* Column `client_location` and `shape` hold the same data 
* `:id` and `:version` are in a corropted state, hence will be ignored

In [37]:
imp_cols = ['eviction_id', 'address', 'city', 'state', 'zip', 'file_date', 'non_payment', 'breach', 'nuisance', 'illegal_use', 'failure_to_sign_renewal', 'access_denial', 'unapproved_subtenant', 'owner_move_in', 'demolition', 'capital_improvement', 'substantial_rehab', 'ellis_act_withdrawal', 'condo_conversion', 'roommate_same_unit', 'other_cause', 'late_payments', 'lead_remediation', 'development', 'good_samaritan_ends', 'constraints_date', 'supervisor_district', 'neighborhood', 'client_location', ':created_at', ':updated_at']


In [38]:
# new df with only selected columns 
df_clean = df.select(imp_cols)
df_clean.show(5)

+-----------+--------------------+-------------+-----+-----+--------------------+-----------+------+--------+-----------+-----------------------+-------------+--------------------+-------------+----------+-------------------+-----------------+--------------------+----------------+------------------+-----------+-------------+----------------+-----------+-------------------+----------------+-------------------+---------------+--------------------+--------------------+--------------------+
|eviction_id|             address|         city|state|  zip|           file_date|non_payment|breach|nuisance|illegal_use|failure_to_sign_renewal|access_denial|unapproved_subtenant|owner_move_in|demolition|capital_improvement|substantial_rehab|ellis_act_withdrawal|condo_conversion|roommate_same_unit|other_cause|late_payments|lead_remediation|development|good_samaritan_ends|constraints_date|supervisor_district|   neighborhood|     client_location|         :created_at|         :updated_at|
+-----------+---

In [39]:
df_clean.printSchema()

root
 |-- eviction_id: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- file_date: string (nullable = true)
 |-- non_payment: boolean (nullable = true)
 |-- breach: boolean (nullable = true)
 |-- nuisance: boolean (nullable = true)
 |-- illegal_use: boolean (nullable = true)
 |-- failure_to_sign_renewal: boolean (nullable = true)
 |-- access_denial: boolean (nullable = true)
 |-- unapproved_subtenant: boolean (nullable = true)
 |-- owner_move_in: boolean (nullable = true)
 |-- demolition: boolean (nullable = true)
 |-- capital_improvement: boolean (nullable = true)
 |-- substantial_rehab: boolean (nullable = true)
 |-- ellis_act_withdrawal: boolean (nullable = true)
 |-- condo_conversion: boolean (nullable = true)
 |-- roommate_same_unit: boolean (nullable = true)
 |-- other_cause: boolean (nullable = true)
 |-- late_payments: boolean (nullable = true)
 |-- le

In [45]:
df_clean.schema

StructType([StructField('eviction_id', StringType(), True), StructField('address', StringType(), True), StructField('city', StringType(), True), StructField('state', StringType(), True), StructField('zip', StringType(), True), StructField('file_date', StringType(), True), StructField('non_payment', BooleanType(), True), StructField('breach', BooleanType(), True), StructField('nuisance', BooleanType(), True), StructField('illegal_use', BooleanType(), True), StructField('failure_to_sign_renewal', BooleanType(), True), StructField('access_denial', BooleanType(), True), StructField('unapproved_subtenant', BooleanType(), True), StructField('owner_move_in', BooleanType(), True), StructField('demolition', BooleanType(), True), StructField('capital_improvement', BooleanType(), True), StructField('substantial_rehab', BooleanType(), True), StructField('ellis_act_withdrawal', BooleanType(), True), StructField('condo_conversion', BooleanType(), True), StructField('roommate_same_unit', BooleanType(

## TESTING: Convert few rows to pandas df and extract the schema

In [2]:
import pandas as p
source_path_json = '/home/sanyashireen/sf_eviction/data_eviction/2023/3/26/raw_eviction_2023-03-26.json'
pandas_df = p.read_json(source_path_json)

In [3]:
pandas_df.shape

(177578, 49)

In [7]:
pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177578 entries, 0 to 177577
Data columns (total 49 columns):
 #   Column                       Non-Null Count   Dtype              
---  ------                       --------------   -----              
 0   :id                          177578 non-null  object             
 1   :created_at                  177578 non-null  datetime64[ns, UTC]
 2   :updated_at                  177578 non-null  datetime64[ns, UTC]
 3   :version                     177578 non-null  object             
 4   :@computed_region_6qbp_sg9q  170478 non-null  float64            
 5   :@computed_region_qgnn_b9vv  171975 non-null  float64            
 6   :@computed_region_26cr_cadq  172007 non-null  float64            
 7   :@computed_region_ajp5_b2md  172007 non-null  float64            
 8   :@computed_region_fyvs_ahh9  172007 non-null  float64            
 9   :@computed_region_p5aj_wyqh  171963 non-null  float64            
 10  :@computed_region_rxqg_mtj9  172

In [9]:
pandas_df.describe().T

,count,mean,std,min,25%,50%,75%,max
:@computed_region_6qbp_sg9q,170478.0,53.222721,33.097547,1.0,25.0,49.0,84.0,117.0
:@computed_region_qgnn_b9vv,171975.0,6.073156,2.814950,1.0,3.0,6.0,9.0,10.0
:@computed_region_26cr_cadq,172007.0,6.118373,3.222694,1.0,3.0,6.0,9.0,11.0
:@computed_region_ajp5_b2md,172007.0,21.027487,11.610458,1.0,11.0,20.0,32.0,41.0
:@computed_region_fyvs_ahh9,172007.0,20.653311,11.547800,1.0,11.0,19.0,32.0,41.0
:@computed_region_p5aj_wyqh,171963.0,5.666736,2.613061,1.0,4.0,6.0,8.0,10.0
:@computed_region_rxqg_mtj9,172007.0,6.306575,3.172408,1.0,4.0,7.0,9.0,11.0
:@computed_region_yftq_j783,171775.0,7.649280,4.847770,1.0,2.0,8.0,11.0,15.0
:@computed_region_bh8s_q3mv,172059.0,18243.866424,13968.876520,54.0,64.0,28853.0,28861.0,29492.0
:@computed_region_6pnf_4xz7,172007.0,1.412443,0.492275,1.0,1.0,1.0,2.0,2.0


In [14]:
pandas_df.head(10)

,:id,:created_at,:updated_at,:version,:@computed_region_6qbp_sg9q,:@computed_region_qgnn_b9vv,:@computed_region_26cr_cadq,:@computed_region_ajp5_b2md,:@computed_region_fyvs_ahh9,:@computed_region_p5aj_wyqh,...,good_samaritan_ends,supervisor_district,neighborhood,client_location,shape,:@computed_region_9jxd_iqea,:@computed_region_6ezc_tdp2,:@computed_region_h4ep_8xdi,:@computed_region_pigm_ib2e,constraints_date
0,row-ufzj_22gk~drek,2023-01-23 23:46:16.858000+00:00,2023-01-23 23:46:33.925000+00:00,rv-wryr~aezj~xte7,53.0,3.0,2.0,20.0,19.0,4.0,...,False,9.0,Mission,"{'latitude': '37.75649855484188', 'longitude':...","{'type': 'Point', 'coordinates': [-122.41447, ...",NaN,NaN,NaN,NaN,NaN
1,row-k4ar_jqfk~qf9p,2023-01-23 23:46:16.858000+00:00,2023-01-23 23:46:33.925000+00:00,rv-jrf5.m8q7.bb2j,55.0,2.0,9.0,26.0,29.0,3.0,...,False,10.0,Potrero Hill,"{'latitude': '37.7636861675663', 'longitude': ...","{'type': 'Point', 'coordinates': [-122.38873, ...",14.0,NaN,NaN,NaN,NaN
2,row-g7er.wz7c~98am,2023-01-23 23:46:16.858000+00:00,2023-01-23 23:46:33.925000+00:00,rv-kub4-qfcm-vv5s,20.0,6.0,10.0,36.0,36.0,1.0,...,False,3.0,Tenderloin,"{'latitude': '37.78620563104688', 'longitude':...","{'type': 'Point', 'coordinates': [-122.41727, ...",NaN,NaN,NaN,NaN,NaN
3,row-jvdz~8ne5_agq5,2023-01-23 23:46:16.858000+00:00,2023-01-23 23:46:33.925000+00:00,rv-t6vv~sqbg.fcxv,55.0,2.0,9.0,26.0,29.0,3.0,...,False,10.0,Potrero Hill,"{'latitude': '37.7636861675663', 'longitude': ...","{'type': 'Point', 'coordinates': [-122.38873, ...",14.0,NaN,NaN,NaN,NaN
4,row-er7z~7fk9~55mx,2023-01-23 23:46:16.858000+00:00,2023-01-23 23:46:33.925000+00:00,rv-8f6e~9dpn.id6z,39.0,10.0,7.0,35.0,35.0,8.0,...,False,4.0,Sunset/Parkside,"{'latitude': '37.748396041406835', 'longitude'...","{'type': 'Point', 'coordinates': [-122.50294, ...",NaN,NaN,NaN,NaN,NaN
5,row-gr32-umab-fxqu,2023-01-23 23:46:16.858000+00:00,2023-01-23 23:46:33.925000+00:00,rv-d76w-xwwa~2fpx,91.0,2.0,2.0,25.0,28.0,3.0,...,False,9.0,Portola,"{'latitude': '37.730376135010886', 'longitude'...","{'type': 'Point', 'coordinates': [-122.40584, ...",NaN,NaN,NaN,NaN,NaN
6,row-5jy6_s62c~m2g4,2023-01-23 23:46:16.858000+00:00,2023-01-23 23:46:33.925000+00:00,rv-pbei~25pq-sufa,44.0,10.0,8.0,41.0,40.0,8.0,...,False,7.0,West of Twin Peaks,"{'latitude': '37.74321534793589', 'longitude':...","{'type': 'Point', 'coordinates': [-122.47192, ...",NaN,NaN,NaN,NaN,NaN
7,row-vzi9_7svc~98jq,2023-01-23 23:46:16.858000+00:00,2023-01-23 23:46:33.925000+00:00,rv-md38~jqdr~y94e,20.0,5.0,10.0,36.0,36.0,10.0,...,False,5.0,Tenderloin,"{'latitude': '37.78333200817521', 'longitude':...","{'type': 'Point', 'coordinates': [-122.40984, ...",6.0,1.0,1.0,18.0,NaN
8,row-hin8.uhzw.rh2d,2023-01-23 23:46:16.858000+00:00,2023-01-23 23:46:33.925000+00:00,rv-r87s.ubap-ypha,20.0,5.0,10.0,36.0,36.0,10.0,...,False,5.0,Tenderloin,"{'latitude': '37.78361971423206', 'longitude':...","{'type': 'Point', 'coordinates': [-122.41506, ...",6.0,1.0,1.0,18.0,NaN
9,row-c8et.y7q5~zf5a,2023-01-23 23:46:16.858000+00:00,2023-01-23 23:46:33.925000+00:00,rv-hhns.jcpx_zh4i,20.0,5.0,10.0,36.0,36.0,10.0,...,False,5.0,Tenderloin,"{'latitude': '37.78361971423206', 'longitude':...","{'type': 'Point', 'coordinates': [-122.41506, ...",6.0,1.0,1.0,18.0,NaN


In [ ]:
### Finding candidates to use as partition column
df[]

In [16]:
# Removing rows with null data so we can avoid the error when convering to spark df for schema purpose
"""The command pandas_df[pandas_df.notnull().all(1)] is used in Python's pandas library to filter rows from a pandas DataFrame based on whether they contain only non-null values in all columns.

Here's how the command works:

pandas_df refers to a pandas DataFrame.
pandas_df.notnull() returns a DataFrame of the same shape as pandas_df but with boolean values indicating whether each element is null or not.
pandas_df.notnull().all(1) returns a Series of boolean values indicating whether all the values in each row of the DataFrame are non-null. The 1 argument specifies that we want to check for non-null values across each row.
Finally, pandas_df[pandas_df.notnull().all(1)] returns a new DataFrame that contains only the rows where all the values are non-null. This is achieved by using the boolean Series to select only the rows where all the values are True.
In summary, this command can be used to filter out rows with missing values from a pandas DataFrame."""
pandas_df= pandas_df[pandas_df.notnull().all(1)]
pandas_df.shape

(12, 49)

In [17]:
# Extract the schema of the pandas df by converting it to pyspark df
spark.createDataFrame(pandas_df).schema
# Modify and align the schema in VSCode

StructType([StructField(':id', StringType(), True), StructField(':created_at', TimestampType(), True), StructField(':updated_at', TimestampType(), True), StructField(':version', StringType(), True), StructField(':@computed_region_6qbp_sg9q', DoubleType(), True), StructField(':@computed_region_qgnn_b9vv', DoubleType(), True), StructField(':@computed_region_26cr_cadq', DoubleType(), True), StructField(':@computed_region_ajp5_b2md', DoubleType(), True), StructField(':@computed_region_fyvs_ahh9', DoubleType(), True), StructField(':@computed_region_p5aj_wyqh', DoubleType(), True), StructField(':@computed_region_rxqg_mtj9', DoubleType(), True), StructField(':@computed_region_yftq_j783', DoubleType(), True), StructField(':@computed_region_bh8s_q3mv', DoubleType(), True), StructField(':@computed_region_6pnf_4xz7', DoubleType(), True), StructField(':@computed_region_jwn9_ihcz', DoubleType(), True), StructField('eviction_id', StringType(), True), StructField('address', StringType(), True), Struc

In [ ]:
pandas_df[pandas_df['illegal_use']==False].shape

In [53]:
pandas_df['illegal_use'].isnull().sum()

0

In [61]:
pandas_df[pandas_df['illegal_use']==True]['illegal_use']

Series([], Name: illegal_use, dtype: bool)

In [35]:
pandas_df[pandas_df['development']=='M221207']

,client_location,neighborhood,non_payment,nuisance,breach,city,address,demolition,illegal_use,access_denial,...,good_samaritan_ends,lead_remediation,unapproved_subtenant,capital_improvement,condo_conversion,late_payments,zip,supervisor_district,owner_move_in,failure_to_sign_renewal


In [62]:
pandas_df['constraints_date']

22026     2011-05-08T00:00:00.000
22039     2011-05-06T00:00:00.000
25330     2009-07-03T00:00:00.000
66403     2011-05-08T00:00:00.000
66416     2011-05-06T00:00:00.000
69707     2009-07-03T00:00:00.000
110783    2011-05-08T00:00:00.000
110794    2011-05-06T00:00:00.000
114085    2009-07-03T00:00:00.000
155227    2011-05-08T00:00:00.000
155238    2011-05-06T00:00:00.000
158529    2009-07-03T00:00:00.000
Name: constraints_date, dtype: object

## Creating PySpark Dataframe by passing the schema
* Refer to the dtypes [here](https://data.sfgov.org/Housing-and-Buildings/Eviction-Notices/5cei-gny5)

In [59]:
from pyspark.sql import types
schema = types.StructType([
    types.StructField(':@computed_region_26cr_cadq', types.StringType(), True), 
    types.StructField(':@computed_region_6ezc_tdp2', types.StringType(), True), 
    types.StructField(':@computed_region_6pnf_4xz7', types.StringType(), True), 
    types.StructField(':@computed_region_6qbp_sg9q', types.StringType(), True), 
    types.StructField(':@computed_region_9jxd_iqea', types.StringType(), True), 
    types.StructField(':@computed_region_ajp5_b2md', types.StringType(), True), 
    types.StructField(':@computed_region_bh8s_q3mv', types.StringType(), True), 
    types.StructField(':@computed_region_fyvs_ahh9', types.StringType(), True), 
    types.StructField(':@computed_region_h4ep_8xdi', types.StringType(), True), 
    types.StructField(':@computed_region_jwn9_ihcz', types.StringType(), True), 
    types.StructField(':@computed_region_p5aj_wyqh', types.StringType(), True), 
    types.StructField(':@computed_region_pigm_ib2e', types.StringType(), True), 
    types.StructField(':@computed_region_qgnn_b9vv', types.StringType(), True), 
    types.StructField(':@computed_region_rxqg_mtj9', types.StringType(), True), 
    types.StructField(':@computed_region_yftq_j783', types.StringType(), True), 
    types.StructField(':created_at', types.TimestampType(), True), 
    types.StructField(':id', types.StringType(), True), 
    types.StructField(':updated_at', types.TimestampType(), True), 
    types.StructField(':version', types.StringType(), True), 
    types.StructField('access_denial', types.BooleanType(), True), 
    types.StructField('address', types.StringType(), True), 
    types.StructField('breach', types.BooleanType(), True), 
    types.StructField('capital_improvement', types.BooleanType(), True), 
    types.StructField('city', types.StringType(), True), 
    types.StructField('client_location', types.MapType(types.StringType(), types.StringType(), True), True), 
    types.StructField('condo_conversion', types.BooleanType(), True), 
    types.StructField('constraints_date', types.StringType(), True), 
    types.StructField('demolition', types.BooleanType(), True), 
    types.StructField('development', types.BooleanType(), True), 
    types.StructField('ellis_act_withdrawal', types.BooleanType(), True), 
    types.StructField('eviction_id', types.StringType(), True), 
    types.StructField('failure_to_sign_renewal', types.BooleanType(), True), 
    types.StructField('file_date', types.DateType(), True), 
    types.StructField('good_samaritan_ends', types.BooleanType(), True), 
    types.StructField('illegal_use', types.BooleanType(), True), 
    types.StructField('late_payments', types.BooleanType(), True), 
    types.StructField('lead_remediation', types.BooleanType(), True), 
    types.StructField('neighborhood', types.StringType(), True), 
    types.StructField('non_payment', types.BooleanType(), True), 
    types.StructField('nuisance', types.BooleanType(), True), 
    types.StructField('other_cause', types.BooleanType(), True), 
    types.StructField('owner_move_in', types.BooleanType(), True), 
    types.StructField('roommate_same_unit', types.BooleanType(), True), 
    types.StructField('shape', types.StructType([
        types.StructField('coordinates', types.ArrayType(types.DoubleType(), True), True), 
        types.StructField('type', types.StringType(), True)]), True), 
    types.StructField('state', types.StringType(), True), 
    types.StructField('substantial_rehab', types.BooleanType(), True), 
    types.StructField('supervisor_district', types.StringType(), True), 
    types.StructField('unapproved_subtenant', types.BooleanType(), True), 
    types.StructField('zip', types.StringType(), True)])

In [60]:
source_path_json = '/home/sanyashireen/sf_eviction/data_eviction/2023/3/25/api_raw_eviction_2023-03-25.json'
raw_pyspark_df = spark.read.option("multiline","true").schema(schema).json(str(source_path_json))

In [150]:
raw_pyspark_df.count()

177578

In [61]:
# Extracting column names to decide which ones are of interest
# Convert type row to dictionary
row_as_dict = raw_pyspark_df.head(1)[0].asDict()

# Sample Column and Value
for key, value in row_as_dict.items():
    print(f'{key}={value}')

:@computed_region_26cr_cadq=2
:@computed_region_6ezc_tdp2=None
:@computed_region_6pnf_4xz7=2
:@computed_region_6qbp_sg9q=53
:@computed_region_9jxd_iqea=None
:@computed_region_ajp5_b2md=20
:@computed_region_bh8s_q3mv=28859
:@computed_region_fyvs_ahh9=19
:@computed_region_h4ep_8xdi=None
:@computed_region_jwn9_ihcz=53
:@computed_region_p5aj_wyqh=4
:@computed_region_pigm_ib2e=None
:@computed_region_qgnn_b9vv=3
:@computed_region_rxqg_mtj9=7
:@computed_region_yftq_j783=2
:created_at=2023-01-23 23:46:16.858000
:id=row-ufzj_22gk~drek
:updated_at=2023-01-23 23:46:33.925000
:version=rv-wryr~aezj~xte7
access_denial=False
address=2500 Block Of Folsom  Street
breach=False
capital_improvement=False
city=San Francisco
client_location={'human_address': '{"address": "", "city": "", "state": "", "zip": ""}', 'latitude': '37.75649855484188', 'longitude': '-122.41446453496935'}
condo_conversion=False
constraints_date=None
demolition=False
development=False
ellis_act_withdrawal=False
eviction_id=M221207
fa

In [43]:
# Extracting column names to decide which ones are of interest
# Convert type row to dictionary
row_as_dict = raw_pyspark_df.head(1)[0].asDict()

# Sample Column and Value
for key, value in row_as_dict.items():
    print(f'{key}={value}')

[':@computed_region_26cr_cadq', ':@computed_region_6ezc_tdp2', ':@computed_region_6pnf_4xz7', ':@computed_region_6qbp_sg9q', ':@computed_region_9jxd_iqea', ':@computed_region_ajp5_b2md', ':@computed_region_bh8s_q3mv', ':@computed_region_fyvs_ahh9', ':@computed_region_h4ep_8xdi', ':@computed_region_jwn9_ihcz', ':@computed_region_p5aj_wyqh', ':@computed_region_pigm_ib2e', ':@computed_region_qgnn_b9vv', ':@computed_region_rxqg_mtj9', ':@computed_region_yftq_j783', ':created_at', ':id', ':updated_at', ':version', 'access_denial', 'address', 'breach', 'capital_improvement', 'city', 'client_location', 'condo_conversion', 'constraints_date', 'demolition', 'development', 'ellis_act_withdrawal', 'eviction_id', 'failure_to_sign_renewal', 'file_date', 'good_samaritan_ends', 'illegal_use', 'late_payments', 'lead_remediation', 'neighborhood', 'non_payment', 'nuisance', 'other_cause', 'owner_move_in', 'roommate_same_unit', 'shape', 'state', 'substantial_rehab', 'supervisor_district', 'unapproved_sub

In [71]:
pyspark_df = raw_pyspark_df.select(imp_cols)

In [72]:
pyspark_df.printSchema()

root
 |-- eviction_id: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: date (nullable = true)
 |-- file_date: string (nullable = true)
 |-- non_payment: boolean (nullable = true)
 |-- breach: boolean (nullable = true)
 |-- nuisance: boolean (nullable = true)
 |-- illegal_use: boolean (nullable = true)
 |-- failure_to_sign_renewal: boolean (nullable = true)
 |-- access_denial: boolean (nullable = true)
 |-- unapproved_subtenant: boolean (nullable = true)
 |-- owner_move_in: boolean (nullable = true)
 |-- demolition: boolean (nullable = true)
 |-- capital_improvement: boolean (nullable = true)
 |-- substantial_rehab: boolean (nullable = true)
 |-- ellis_act_withdrawal: boolean (nullable = true)
 |-- condo_conversion: boolean (nullable = true)
 |-- roommate_same_unit: boolean (nullable = true)
 |-- other_cause: boolean (nullable = true)
 |-- late_payments: boolean (nullable = true)
 |-- lead

## Option 1: Write partitioned data to local
* And then use the prefect blocks to write the data to GCS and BQ

In [36]:
# folder to write the partition on the local system
data_partition_dir = '/home/sanyashireen/sf_eviction/data_eviction/2023/3/22/clean_eviction_2023-03-22_partitioned'
pyspark_df = pyspark_df.repartition(100)
pyspark_df.write.parquet(data_partition_dir, mode='overwrite')

23/03/23 18:48:08 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 45, schema size: 29
CSV file: file:///home/sanyashireen/sf_eviction/data_eviction/2023/3/22/gcs_raw_eviction_2023-03-22.csv


## Option 2: Write partitioned data directly to gcs and bq (as an external table)
* Will need to use Dataproc for that.

In [109]:
from pyspark.sql.functions import col, expr

In [158]:
spark.stop()

In [88]:
from pyspark.sql.functions import col
source_path_json = '/home/sanyashireen/sf_eviction/data_eviction/2023/3/25/api_raw_eviction_2023-03-25.json'
data_dir = Path('/home/sanyashireen/sf_eviction/data_eviction/2023/3/25')
target_dir = '/home/sanyashireen/sf_eviction/data_eviction/2023/3/25/clean_partitioned_eviction_2023-03-25'

spark = SparkSession.builder \
                    .master("local[*]") \
                    .appName('test') \
                    .getOrCreate()

imp_cols = ['eviction_id', 'address', 'city', 'state', 'zip', 'file_date', 'non_payment', 'breach', 'nuisance', 'illegal_use', 'failure_to_sign_renewal', 'access_denial', 'unapproved_subtenant', 'owner_move_in', 'demolition', 'capital_improvement', 'substantial_rehab', 'ellis_act_withdrawal', 'condo_conversion', 'roommate_same_unit', 'other_cause', 'late_payments', 'lead_remediation', 'development', 'good_samaritan_ends', 'constraints_date', 'supervisor_district', 'neighborhood', 'client_location', ':created_at', ':updated_at']
schema = schema = types.StructType([
    types.StructField(':@computed_region_26cr_cadq', types.StringType(), True), 
    types.StructField(':@computed_region_6ezc_tdp2', types.StringType(), True), 
    types.StructField(':@computed_region_6pnf_4xz7', types.StringType(), True), 
    types.StructField(':@computed_region_6qbp_sg9q', types.StringType(), True), 
    types.StructField(':@computed_region_9jxd_iqea', types.StringType(), True), 
    types.StructField(':@computed_region_ajp5_b2md', types.StringType(), True), 
    types.StructField(':@computed_region_bh8s_q3mv', types.StringType(), True), 
    types.StructField(':@computed_region_fyvs_ahh9', types.StringType(), True), 
    types.StructField(':@computed_region_h4ep_8xdi', types.StringType(), True), 
    types.StructField(':@computed_region_jwn9_ihcz', types.StringType(), True), 
    types.StructField(':@computed_region_p5aj_wyqh', types.StringType(), True), 
    types.StructField(':@computed_region_pigm_ib2e', types.StringType(), True), 
    types.StructField(':@computed_region_qgnn_b9vv', types.StringType(), True), 
    types.StructField(':@computed_region_rxqg_mtj9', types.StringType(), True), 
    types.StructField(':@computed_region_yftq_j783', types.StringType(), True), 
    types.StructField(':created_at', types.TimestampType(), True), 
    types.StructField(':id', types.StringType(), True), 
    types.StructField(':updated_at', types.TimestampType(), True), 
    types.StructField(':version', types.StringType(), True), 
    types.StructField('access_denial', types.BooleanType(), True), 
    types.StructField('address', types.StringType(), True), 
    types.StructField('breach', types.BooleanType(), True), 
    types.StructField('capital_improvement', types.BooleanType(), True), 
    types.StructField('city', types.StringType(), True), 
    types.StructField('client_location', types.MapType(types.StringType(), types.StringType(), True), True), 
    types.StructField('condo_conversion', types.BooleanType(), True), 
    types.StructField('constraints_date', types.StringType(), True), 
    types.StructField('demolition', types.BooleanType(), True), 
    types.StructField('development', types.BooleanType(), True), 
    types.StructField('ellis_act_withdrawal', types.BooleanType(), True), 
    types.StructField('eviction_id', types.StringType(), True), 
    types.StructField('failure_to_sign_renewal', types.BooleanType(), True), 
    types.StructField('file_date', types.DateType(), True), 
    types.StructField('good_samaritan_ends', types.BooleanType(), True), 
    types.StructField('illegal_use', types.BooleanType(), True), 
    types.StructField('late_payments', types.BooleanType(), True), 
    types.StructField('lead_remediation', types.BooleanType(), True), 
    types.StructField('neighborhood', types.StringType(), True), 
    types.StructField('non_payment', types.BooleanType(), True), 
    types.StructField('nuisance', types.BooleanType(), True), 
    types.StructField('other_cause', types.BooleanType(), True), 
    types.StructField('owner_move_in', types.BooleanType(), True), 
    types.StructField('roommate_same_unit', types.BooleanType(), True), 
    types.StructField('shape', types.StructType([
        types.StructField('coordinates', types.ArrayType(types.DoubleType(), True), True), 
        types.StructField('type', types.StringType(), True)]), True), 
    types.StructField('state', types.StringType(), True), 
    types.StructField('substantial_rehab', types.BooleanType(), True), 
    types.StructField('supervisor_district', types.StringType(), True), 
    types.StructField('unapproved_subtenant', types.BooleanType(), True), 
    types.StructField('zip', types.StringType(), True)])

df = spark.read \
          .option("multiline","true") \
          .schema(schema) \
          .json(str(source_path_json)) \
          .select(imp_cols) \
          .withColumnRenamed(':updated_at', 'updated_at') \
          .withColumnRenamed(':created_at', 'created_at') 
print(f'Total rows read: {df.count()}')
df = df.withColumn("latitude", df["client_location"].getItem("latitude").cast("double")) \
       .withColumn("longitude", df["client_location"].getItem("longitude").cast("double")) \
       .drop("client_location")
df = df.repartition(2)

df.write.parquet(target_dir, mode='overwrite')
spark.stop()

Total rows read: 177578


## testing


In [86]:
df.schema
  

StructType([StructField('eviction_id', StringType(), True), StructField('address', StringType(), True), StructField('city', StringType(), True), StructField('state', StringType(), True), StructField('zip', StringType(), True), StructField('file_date', DateType(), True), StructField('non_payment', BooleanType(), True), StructField('breach', BooleanType(), True), StructField('nuisance', BooleanType(), True), StructField('illegal_use', BooleanType(), True), StructField('failure_to_sign_renewal', BooleanType(), True), StructField('access_denial', BooleanType(), True), StructField('unapproved_subtenant', BooleanType(), True), StructField('owner_move_in', BooleanType(), True), StructField('demolition', BooleanType(), True), StructField('capital_improvement', BooleanType(), True), StructField('substantial_rehab', BooleanType(), True), StructField('ellis_act_withdrawal', BooleanType(), True), StructField('condo_conversion', BooleanType(), True), StructField('roommate_same_unit', BooleanType(),

In [87]:
df.show(5)

+-----------+--------------------+-------------+-----+-----+----------+-----------+------+--------+-----------+-----------------------+-------------+--------------------+-------------+----------+-------------------+-----------------+--------------------+----------------+------------------+-----------+-------------+----------------+-----------+-------------------+----------------+-------------------+--------------------+--------------------+--------------------+------------------+-------------------+
|eviction_id|             address|         city|state|  zip| file_date|non_payment|breach|nuisance|illegal_use|failure_to_sign_renewal|access_denial|unapproved_subtenant|owner_move_in|demolition|capital_improvement|substantial_rehab|ellis_act_withdrawal|condo_conversion|roommate_same_unit|other_cause|late_payments|lead_remediation|development|good_samaritan_ends|constraints_date|supervisor_district|        neighborhood|          created_at|          updated_at|          latitude|          l

In [164]:
df.schema

StructType([StructField(':@computed_region_26cr_cadq', StringType(), True), StructField(':@computed_region_6ezc_tdp2', StringType(), True), StructField(':@computed_region_6pnf_4xz7', StringType(), True), StructField(':@computed_region_6qbp_sg9q', StringType(), True), StructField(':@computed_region_9jxd_iqea', StringType(), True), StructField(':@computed_region_ajp5_b2md', StringType(), True), StructField(':@computed_region_bh8s_q3mv', StringType(), True), StructField(':@computed_region_fyvs_ahh9', StringType(), True), StructField(':@computed_region_h4ep_8xdi', StringType(), True), StructField(':@computed_region_jwn9_ihcz', StringType(), True), StructField(':@computed_region_p5aj_wyqh', StringType(), True), StructField(':@computed_region_pigm_ib2e', StringType(), True), StructField(':@computed_region_qgnn_b9vv', StringType(), True), StructField(':@computed_region_rxqg_mtj9', StringType(), True), StructField(':@computed_region_yftq_j783', StringType(), True), StructField(':created_at', S

In [98]:
from pyspark.sql.functions import explode
df.select(df.client_location,explode(df.client_location)).show()

+--------------------+-------------+--------------------+
|     client_location|          key|               value|
+--------------------+-------------+--------------------+
|{latitude -> 37.7...|     latitude|  37.757015976704615|
|{latitude -> 37.7...|    longitude|  -122.4357441652753|
|{latitude -> 37.7...|human_address|{"address": "", "...|
|{latitude -> 37.7...|     latitude|  37.757015976704615|
|{latitude -> 37.7...|    longitude|  -122.4357441652753|
|{latitude -> 37.7...|human_address|{"address": "", "...|
|{latitude -> 37.7...|     latitude|  37.757015976704615|
|{latitude -> 37.7...|    longitude|  -122.4357441652753|
|{latitude -> 37.7...|human_address|{"address": "", "...|
|{latitude -> 37.7...|     latitude|  37.757015976704615|
|{latitude -> 37.7...|    longitude|  -122.4357441652753|
|{latitude -> 37.7...|human_address|{"address": "", "...|
|{latitude -> 37.7...|     latitude|  37.757015976704615|
|{latitude -> 37.7...|    longitude|  -122.4357441652753|
|{latitude -> 